# Analysis of Predictions Produced on Model Level with RNNModel Class by Darts

This script analyzes all pickle files in `./data/{approach}/{n_chunks}_chunks/{style}/`, starting with `confusion_matrix_models`, i.e. all model results. At the moment, the paths are adapted for local execution.

## Merge and Adjust Model Results

### Define Variables to Adjust

In [ ]:
# Define approach
approach = 'RNNModel'

# Define number of chunks (1000, 2000 or 15000)
n_chunks = 2000

# Define how many chunks were taken for prediction ('all' or '20_percent')
style = 'all'

# Defines if scaled values should be read and if yes, which scaled values
# '': read all results
# '_n': read non-scaled results
# '_s1': read results produced with series scaled using standard score (same mean and same standard deviation)
# '_s2': read results produced with series scaled separately with several MinMaxScalers
scaling_version = '_s1'

### Extract All Generated Model Results

In [ ]:
import os
import pandas as pd
import pickle5 as pickle

# Define path to all model matrices produced by prediction
path_to_model_matrices = f'../../data/{approach}/{n_chunks}_chunks/{style}'

# Concat all found matrices into result matrix
result_matrix_models = pd.DataFrame(columns=['ID', 'PARAMETER', 'RUNTIME', 'MODEL', 'SCALING', 'LIBRARY', 'ENDOGENOUS',
                                             'EXOGENOUS', 'FIRST_FORECAST', 'ALARM_TYPE', 'FP', 'TP', 'FN', 'TN',
                                             'N_HIGH_ALARMS', 'N_LOW_ALARMS', 'N_CHUNKS', 'N_ITERATIONS'])

for file in os.listdir(path_to_model_matrices):
    if os.path.isfile(os.path.join(path_to_model_matrices, file)) and \
            file.startswith('confusion_matrix_models') and file.endswith(f'{scaling_version}.pickle'):

        # Read file
        current_matrix_f = open(f'{path_to_model_matrices}/{file}', 'rb')
        current_matrix = pickle.load(current_matrix_f)
        current_matrix_f.close()

        # Append current matrix to result matrix
        result_matrix_models = pd.concat([result_matrix_models, current_matrix])

### Add Metrics

See https://en.wikipedia.org/wiki/Sensitivity_and_specificity for more information.

In [ ]:
# Add confusion matrix ratios
result_matrix_models['FPR'] = result_matrix_models['FP'] / (result_matrix_models['FP'] + result_matrix_models['TN'])
result_matrix_models['TPR'] = result_matrix_models['TP'] / (result_matrix_models['TP'] + result_matrix_models['FN'])
result_matrix_models['FNR'] = result_matrix_models['FN'] / (result_matrix_models['TP'] + result_matrix_models['FN'])
result_matrix_models['TNR'] = result_matrix_models['TN'] / (result_matrix_models['FP'] + result_matrix_models['TN'])

# Add accuracy
result_matrix_models['ACC'] = (result_matrix_models['TP'] + result_matrix_models['TN']) / \
                              (result_matrix_models['TP'] + result_matrix_models['FN']
                               + result_matrix_models['FP'] + result_matrix_models['TN'])

# Add F1 score (harmonic mean of precision/PPV and sensitivity/TPR)
result_matrix_models['F1S'] = result_matrix_models['TP'] / \
                              (result_matrix_models['TP'] +
                               0.5 * (result_matrix_models['FP'] + result_matrix_models['FN']))

# Add threat score
result_matrix_models['TS'] = result_matrix_models['TP'] / (result_matrix_models['TP'] + result_matrix_models['FN'] + result_matrix_models['FP'])

# Add Matthews correlation coefficient
result_matrix_models['MCC_divident'] = result_matrix_models['TP'] * result_matrix_models['TN'] - \
                                       result_matrix_models['FP'] * result_matrix_models['FN']
result_matrix_models['MCC_divisor'] = ((result_matrix_models['TP'] + result_matrix_models['FP']) *
                                              (result_matrix_models['TP'] + result_matrix_models['FN']) *
                                              (result_matrix_models['TN'] + result_matrix_models['FP']) *
                                              (result_matrix_models['TN'] + result_matrix_models['FN']))**(1/2)
result_matrix_models['MCC'] = result_matrix_models['MCC_divident'] / result_matrix_models['MCC_divisor']

# Add weighted score from https://physionet.org/content/challenge-2015/1.0.0/
# Original: (TP + TN) / (TP + TN + FP + 5*FN)
# Adapted: (TP) / (TP + FN + 5*FP)
result_matrix_models['WEIGHTED_SCORE_AA'] = result_matrix_models['TP'] / \
                                            (result_matrix_models['TP'] + result_matrix_models['FN'] +
                                             (5 * result_matrix_models['FP']))

# Round all floats to 4 decimal places
# Note: round() does not work for floats with many decimal places
decimals = 4
for col in ['FPR', 'TPR', 'FNR', 'TNR', 'ACC', 'F1S', 'TS', 'MCC', 'WEIGHTED_SCORE_AA']:
    result_matrix_models[col] = result_matrix_models[col].apply(lambda x: round(x, decimals))

# Move cols to end for similarity with ARIMA results and drop columns (alarm counts and MCC helper columns)
result_matrix_models = result_matrix_models[['ID', 'PARAMETER', 'RUNTIME', 'MODEL', 'SCALING', 'LIBRARY', 'ENDOGENOUS',
                                             'EXOGENOUS', 'FIRST_FORECAST', 'ALARM_TYPE', 'TP', 'FN', 'FP', 'TN',
                                             'FPR', 'TPR', 'FNR', 'TNR', 'ACC', 'F1S', 'MCC', 'TS', 'WEIGHTED_SCORE_AA',
                                             'N_CHUNKS', 'N_ITERATIONS']]

### Add Temporary Missing Rows

This addition is made to view results while scripts are still running.

In [ ]:
import numpy as np

# Get missing IDs
if scaling_version == '':
    scaling_suffixes = ['_n', '_s1', '_s2']
else:
    scaling_suffixes = [scaling_version]

all_ids = list()
for param in ['BP', 'HR', 'O2']:
    for model_prefix in ['RN', 'LS', 'GR']:
        for scaling_suffix in scaling_suffixes:
            all_ids.append(f'{param}_{model_prefix}_01_{scaling_suffix}_H')
            all_ids.append(f'{param}_{model_prefix}_01_{scaling_suffix}_L')

            all_ids.append(f'{param}_{model_prefix}_02_{scaling_suffix}_H')
            all_ids.append(f'{param}_{model_prefix}_02_{scaling_suffix}_L')

missing_ids = list(set(all_ids).difference(set(pd.unique(result_matrix_models.ID))))
print(f'Missing IDs: {missing_ids}')

if len(missing_ids) > 0:

    # Add missing rows with columns that are not row-specific
    missing_rows = pd.DataFrame(data={'ID': list(missing_ids),
                                      'LIBRARY' : ['darts'] * len(missing_ids),
                                      'FIRST_FORECAST' : [12] * len(missing_ids)})

    # Add row-specific columns
    missing_rows['PARAMETER'] = missing_rows['ID'].str[:2]
    missing_rows['ALARM_TYPE'] = ['Low' if model_id[-1] == 'L' else 'High' for model_id in missing_rows['ID'] ]

    missing_rows['SCALING'] = ['Standard' if model_id.split('_')[3] == 's1'
                               else 'Min-Max' if model_id.split('_')[3] == 's2'
                               else np.nan for model_id in missing_rows['ID']]

    missing_rows['MODEL'] = ['RNN' if model_id.split('_')[1] == 'RN'
                             else 'LSTM' if model_id.split('_')[1] == 'LS'
                             else 'GRU' for model_id in missing_rows['ID']]

    missing_rows['ENDOGENOUS'] = ['Median' if model_id.split('_')[2] == '01'
                                  else 'Max' if model_id.split('_')[2] == '02' and model_id[-1] == 'H'
                                  else 'Min' for model_id in missing_rows['ID']]

    missing_rows['EXOGENOUS'] = [np.nan if model_id.split('_')[2] == '01'
                                 else 'Median' for model_id in missing_rows['ID']]

    # Add missing rows (and fill missing columns with NaN by default)
    result_matrix_models = pd.concat([result_matrix_models, missing_rows], axis=0, ignore_index=True)

### Finalize and Save as Parquet File

In [ ]:
from IPython.display import display

# Adjust data types
result_matrix_models['FIRST_FORECAST'] = result_matrix_models['FIRST_FORECAST'].astype(int)

if len(missing_ids) == 0:
    result_matrix_models['RUNTIME'] = result_matrix_models['RUNTIME'].astype(int)
    result_matrix_models['TP'] = result_matrix_models['TP'].astype(int)
    result_matrix_models['TN'] = result_matrix_models['TN'].astype(int)
    result_matrix_models['FP'] = result_matrix_models['FP'].astype(int)
    result_matrix_models['FN'] = result_matrix_models['FN'].astype(int)

# Sort result matrix for better readability
result_matrix_models.sort_values(by=['ID'], inplace=True)

# Reset index
result_matrix_models.reset_index(inplace=True, drop=True)

# Show result matrix per parameter
display(result_matrix_models[result_matrix_models['PARAMETER'] == 'BP'])
display(result_matrix_models[result_matrix_models['PARAMETER'] == 'HR'])
display(result_matrix_models[result_matrix_models['PARAMETER'] == 'O2'])

# Save as parquet file
result_matrix_models.to_parquet(f'../../data/{approach}/{n_chunks}_chunks/{style}/rnn_model_results_{n_chunks}_{style}'
                                f'{scaling_version}.parquet', engine='pyarrow')

### Combine Different Result Matrices

This step is only needed if separate result matrix files were created and should be combined for visualization.

In [ ]:
# Read non-scaled results
results_normal = pd.read_parquet(f'../../data/{approach}/{n_chunks}_chunks/{style}/rnn_model_results_{n_chunks}_{style}'
                                 f'_n.parquet', engine='pyarrow')

# Read standard score-scaled results
results_standard_scaled = pd.read_parquet(f'../../data/{approach}/{n_chunks}_chunks/{style}/rnn_model_results_{n_chunks}_'
                                   f'{style}_s1.parquet', engine='pyarrow')

# Read min-max-scaled results
results_min_max_scaled = pd.read_parquet(f'../../data/{approach}/{n_chunks}_chunks/{style}/rnn_model_results_{n_chunks}_'
                                         f'{style}_s2.parquet', engine='pyarrow')

# Concat all result matrices
rnn_model_results = pd.concat([results_normal, results_standard_scaled, results_min_max_scaled],
                              axis=0,
                              ignore_index=True)

# Sort result matrix for better readability
rnn_model_results.sort_values(by=['ID'], inplace=True)

# Reset index
rnn_model_results.reset_index(inplace=True, drop=True)

# Show result matrix per parameter
display(rnn_model_results[rnn_model_results['PARAMETER'] == 'BP'])
display(rnn_model_results[rnn_model_results['PARAMETER'] == 'HR'])
display(rnn_model_results[rnn_model_results['PARAMETER'] == 'O2'])

# Save as parquet file
rnn_model_results.to_parquet(f'../../data/{approach}/{n_chunks}_chunks/{style}/rnn_model_results_{n_chunks}_{style}'
                             f'.parquet', engine='pyarrow')

## Visualization of Model Level Results

### Setup Variables for All Model Level Plots

In [ ]:
import numpy as np
import pandas as pd

# Set variables defining path
approach = 'RNNModel'
n_chunks = 2000
style = 'all'

# Suffix can be '', '_n', '_s1' or '_s2'
suffix = ''

# Decide whether plots with only one metric shown should be horizontally or vertically oriented
vertical_oriented = True

# Decide whether plots with only one metric shown should be saved separately per parameter or not
separate_plots = False

In [ ]:
# Read respective file
result_matrix_models = pd.read_parquet(f'../../data/{approach}/{n_chunks}_chunks/{style}/rnn_model_results_{n_chunks}_'
                                       f'{style}{suffix}.parquet', engine='pyarrow')
result_matrix_models.info()

# Avoid error because of non-found values
available_parameters = pd.unique(result_matrix_models.PARAMETER)

# Only plot content for available parameters
n_cols = len(available_parameters)

# If all runs are finished, visualize with custom order
if len(available_parameters) == 3:
    available_parameters = ['HR', 'BP', 'O2']

parameter_labels = {
    'HR' : '$\mathregular{HR}$',
    'BP' : '$\mathregular{NBPs}$',
    'O2' : '$\mathregular{S_pO_2}$'
}

# Decide if parameters are plotted above or beside each other (if only one metric is considered)
if vertical_oriented:
    figure_suffix = '_vertical'
    nr=n_cols
    nc=1
    fs=(6, 14)
else:
    figure_suffix = '_horizontal'
    nr=1
    nc=n_cols
    fs=(20, 4)

### Plot Accuracy, TPR, FNR, and TNR

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# "Group" result matrix by prefix of ID
plotdata = result_matrix_models.copy()
plotdata['ID'] = plotdata['ID'] + '_'
plotdata = plotdata.replace(['_H_', '_L_'], ['', ''], regex=True)

# Create subplots
sns.set_style('whitegrid')
fig, axs = plt.subplots(
    nrows=4,
    ncols=n_cols,
    figsize=(15, 13),
    dpi=72
    )

#plt.suptitle(f'Accuracy, TPR, FNR, and TNR of {n_chunks} Chunks ({style.replace("_", " ").upper()})', fontsize=22)

# Define y-limits
acc_ylimits = [0, result_matrix_models.ACC.max(skipna=True) + 0.05]
tpr_ylimits = [0, result_matrix_models.TPR.max(skipna=True) + 0.05]
fnr_ylimits = [0, result_matrix_models.FNR.max(skipna=True) + 0.05]
tnr_ylimits = [0, result_matrix_models.TNR.max(skipna=True) + 0.05]

# Actual plots
for i, parameter in enumerate(available_parameters):

    sns.barplot(
        ax=axs[0, i],
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='ACC',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None,
        hue_order=['Low', 'High'])
    axs[0, i].set_title(parameter_labels[parameter], fontweight='bold', color='black', fontsize=20)
    axs[0, i].set_ylim(acc_ylimits)
    axs[0, i].set_xticklabels(axs[0, i].get_xticklabels(), rotation=90)
    axs[0, i].legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

    sns.barplot(
        ax=axs[1, i],
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='TPR',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None,
        hue_order=['Low', 'High'])
    axs[1, i].set_ylim(tpr_ylimits)
    axs[1, i].set_xticklabels(axs[1, i].get_xticklabels(), rotation=90)
    axs[1, i].legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

    sns.barplot(
        ax=axs[2, i],
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='FNR',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None,
        hue_order=['Low', 'High'])
    axs[2, i].set_ylim(fnr_ylimits)
    axs[2, i].set_xticklabels(axs[2, i].get_xticklabels(), rotation=90)
    axs[2, i].legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

    sns.barplot(
        ax = axs[3, i],
        data = plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='TNR',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None,
        hue_order=['Low', 'High'])
    axs[3, i].set_ylim(tnr_ylimits)
    axs[3, i].set_xticklabels(axs[3, i].get_xticklabels(), rotation=90)
    axs[3, i].legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

# Improve layout and save figure
fig.tight_layout()
plt.show(fig)
fig.savefig(f'../../plots/{approach}/{n_chunks}_chunks/{style}/rnn_results_tpr_fnr_tnr_acc.pdf', dpi=300)

### Plot False Positive Ratio and F1 Score

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# "Group" result matrix by prefix of ID
plotdata = result_matrix_models.copy()
plotdata['ID'] = plotdata['ID'] + '_'
plotdata = plotdata.replace(['_H_', '_L_'], ['', ''], regex=True)

# Create subplots
sns.set_style('whitegrid')
fig, axs = plt.subplots(
    nrows=2,
    ncols=n_cols,
    figsize=(15, 7),
    dpi=72
    )

#yplt.suptitle(f'FPR and F1S of {n_chunks} Chunks ({style.replace("_", " ").upper()})', fontsize=22)

# Define y-limits
fpr_ylimits = [0, result_matrix_models.FPR.max(skipna=True) + 0.05]
f1s_ylimits = [0, result_matrix_models.F1S.max(skipna=True) + 0.05]

# Actual plot
for i, parameter in enumerate(available_parameters):

    print(f'\n##### {parameter} #####')

    g_fpr = sns.barplot(
        ax=axs[0, i],
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='FPR',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None,
        hue_order=['Low', 'High'])
    axs[0, i].set_title(parameter_labels[parameter], fontweight='bold', color='black', fontsize=14)
    axs[0, i].set_ylim(fpr_ylimits)
    axs[0, i].set_xticklabels(axs[0, i].get_xticklabels(), rotation=90)
    axs[0, i].legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

    # Show models with best FPR
    best_fpr_low = plotdata[plotdata.FPR == plotdata[(plotdata.PARAMETER == parameter) & (plotdata.ALARM_TYPE == 'Low')].FPR.min()]['FPR'].unique()
    print(f'Best low FPR: {best_fpr_low}')
    best_fpr_high = plotdata[plotdata.FPR == plotdata[(plotdata.PARAMETER == parameter) & (plotdata.ALARM_TYPE == 'High')].FPR.min()]['FPR'].unique()
    print(f'Best high FPR: {best_fpr_high}')

    # Add red rectangle around models with best F1 score
    for bar in g_fpr.patches:
        if bar.get_height() == best_fpr_low or bar.get_height() == best_fpr_high:
            bar.set_edgecolor('red')
            bar.set_linewidth(2)

    g_f1s = sns.barplot(
        ax=axs[1, i],
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='F1S',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None,
        hue_order=['Low', 'High'])
    axs[1, i].set_ylim(f1s_ylimits)
    axs[1, i].set_xticklabels(axs[1, i].get_xticklabels(), rotation=90)
    axs[1, i].legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

    # Show models with best F1 scores
    best_f1s_low = plotdata[plotdata.F1S == plotdata[(plotdata.PARAMETER == parameter) & (plotdata.ALARM_TYPE == 'Low')].F1S.max()]['F1S'].unique()
    print(f'Best low F1S: {best_f1s_low}')
    best_f1s_high = plotdata[plotdata.F1S == plotdata[(plotdata.PARAMETER == parameter) & (plotdata.ALARM_TYPE == 'High')].F1S.max()]['F1S'].unique()
    print(f'Best high F1S: {best_f1s_high}')

    # Add red rectangle around models with best F1 score
    for bar in g_f1s.patches:
        if bar.get_height() == best_f1s_low or bar.get_height() == best_f1s_high:
            bar.set_edgecolor('red')
            bar.set_linewidth(2)

# Improve layout and save figure
fig.tight_layout()
plt.show(fig)
fig.savefig(f'../../plots/{approach}/{n_chunks}_chunks/{style}/rnn_results_fpr_f1s.pdf', dpi=300)

### Plot TS, Weighted Score & MCC

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plotdata = result_matrix_models.copy()
plotdata['ID'] = plotdata['ID'] + '_'
plotdata = plotdata.replace(['_H_', '_L_'], ['', ''], regex=True)

sns.set_style('whitegrid')

fig, axs = plt.subplots(
    nrows=3,
    ncols=n_cols,
    figsize=(20, 12),
    dpi=72
    )
fig.subplots_adjust(hspace=0.6, wspace=0.4)

ts_ylimits = [0, result_matrix_models.TS.max(skipna=True) + 0.05]
weighted_aa_score_ylimits = [0, result_matrix_models.WEIGHTED_SCORE_AA.max(skipna=True) + 0.05]
mcc_ylimits = [0, result_matrix_models.MCC.max(skipna=True) + 0.05]

for i, parameter in enumerate(available_parameters):

    print(f'\n##### {parameter} #####')

    g_ts = sns.barplot(
        ax=axs[0, i],
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='TS',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None,
        hue_order=['Low', 'High'])
    axs[0, i].set_title(parameter_labels[parameter], fontweight='bold', color='black', fontsize=14)
    axs[0, i].set_ylim(ts_ylimits)
    axs[0, i].tick_params(axis='x', rotation=90)
    axs[0, i].legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

    # Show models with best TS
    best_ts_low = plotdata[plotdata.TS == plotdata[(plotdata.PARAMETER == parameter) & (plotdata.ALARM_TYPE == 'Low')].TS.max()]['TS'].unique()
    print(f'Best low TS: {best_ts_low}')
    best_ts_high = plotdata[plotdata.TS == plotdata[(plotdata.PARAMETER == parameter) & (plotdata.ALARM_TYPE == 'High')].TS.max()]['TS'].unique()
    print(f'Best high TS: {best_ts_high}')

    # Highlight models with best TS
    for bar in g_ts.patches:
        if bar.get_height() != best_ts_low and bar.get_height() != best_ts_high :
            bar.set_alpha(0.3)

    g_ws_aa = sns.barplot(
        ax=axs[1, i],
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='WEIGHTED_SCORE_AA',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None,
        hue_order=['Low', 'High'])
    axs[1, i].set_ylim(weighted_aa_score_ylimits)
    axs[1, i].tick_params(axis='x', rotation=90)
    axs[1, i].legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

    # Show models with best WS_AA
    best_weighted_aa_score_low = plotdata[plotdata.WEIGHTED_SCORE_AA == plotdata[(plotdata.PARAMETER == parameter) & (plotdata.ALARM_TYPE == 'Low')].WEIGHTED_SCORE_AA.max()]['WEIGHTED_SCORE_AA'].unique()
    print(f'Best low WS_AA: {best_weighted_aa_score_low}')
    best_weighted_aa_score_high = plotdata[plotdata.WEIGHTED_SCORE_AA == plotdata[(plotdata.PARAMETER == parameter) & (plotdata.ALARM_TYPE == 'High')].WEIGHTED_SCORE_AA.max()]['WEIGHTED_SCORE_AA'].unique()
    print(f'Best high WS_AA: {best_weighted_aa_score_high}')

    # Highlight models with best WS_AA
    for bar in g_ws_aa.patches:
        if bar.get_height() != best_weighted_aa_score_low and bar.get_height() != best_weighted_aa_score_high :
            bar.set_alpha(0.3)

    g_mcc = sns.barplot(
        ax=axs[2, i],
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='MCC',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None,
        hue_order=['Low', 'High'])
    axs[2, i].set_ylim(mcc_ylimits)
    axs[2, i].tick_params(axis='x', rotation=90)
    axs[2, i].legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

    # Show models with best MCC
    best_mcc_low = plotdata[plotdata.MCC == plotdata[(plotdata.PARAMETER == parameter) & (plotdata.ALARM_TYPE == 'Low')].MCC.max()]['MCC'].unique()
    print(f'Best low MCC: {best_mcc_low}')
    best_mcc_high = plotdata[plotdata.MCC == plotdata[(plotdata.PARAMETER == parameter) & (plotdata.ALARM_TYPE == 'High')].MCC.max()]['MCC'].unique()
    print(f'Best high MCC: {best_mcc_high}')

    # Highlight models with best MCC
    for bar in g_mcc.patches:
        if bar.get_height() != best_mcc_low and bar.get_height() != best_mcc_high :
            bar.set_alpha(0.3)

# Save figure
plt.savefig(f'../../plots/{approach}/{n_chunks}_chunks/{style}/rnn_results_ts_ws_mcc.pdf', dpi=300, bbox_inches='tight')
plt.show(fig)

### Compare Scaling Methods

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plotdata = result_matrix_models[result_matrix_models['SCALING'] != np.nan]
plotdata = plotdata.replace(['_s1', '_s2'], ['', ''], regex=True)

best_scaling_method_indices = pd.DataFrame(plotdata.groupby(['ID'])['WEIGHTED_SCORE_AA'].idxmax())
winning_scaling_method = plotdata[plotdata.index.isin(best_scaling_method_indices['WEIGHTED_SCORE_AA'].tolist())][['ID', 'SCALING']]
print(winning_scaling_method)

plotdata['TEMPORARY_SORTING_COLUMN'] = plotdata['ID'] + '_'
plotdata['TEMPORARY_SORTING_COLUMN'] = plotdata.TEMPORARY_SORTING_COLUMN.replace(['_H_', '_L_'], ['', ''], regex=True)
plotdata = plotdata.sort_values(by=['TEMPORARY_SORTING_COLUMN', 'ALARM_TYPE'], ascending = [True, False]).drop(['TEMPORARY_SORTING_COLUMN'], axis = 1).reset_index(drop=True)

if not separate_plots:
    sns.set_style('whitegrid')
    fig, axs = plt.subplots(
        nrows=nr,
        ncols=nc,
        figsize=fs,
        dpi=72
        )
    fig.subplots_adjust(hspace=0.6, wspace=0.4)

    evaluation_score_ylimits = [0, result_matrix_models.WEIGHTED_SCORE_AA.max(skipna=True) + 0.05]

for i, parameter in enumerate(available_parameters):
    if separate_plots:
        # Setup plot
        sns.set_style('whitegrid')
        fig, ax = plt.subplots(
            nrows=1,
            ncols=1,
            dpi=72
            )
        fig.subplots_adjust(hspace=0.6, wspace=0.4)

        evaluation_score_ylimits = [0, plotdata[plotdata.PARAMETER == parameter].WEIGHTED_SCORE_AA.max(skipna=True) + 0.05]
        axes = ax

    else:
        axes = axs[i]

    g_evaluation_score = sns.barplot(
        ax=axes,
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='WEIGHTED_SCORE_AA',
        hue='SCALING',
        palette=sns.color_palette('colorblind')[2:],
        ci=None)
    axes.set_title(parameter_labels[parameter], fontweight='bold', color='black', fontsize=14)
    axes.set_ylim(evaluation_score_ylimits)
    axes.tick_params(axis='x', rotation=90)
    axes.set_ylabel('Evaluation Score')
    axes.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, title='Scaling Method: ')

    """
    # Highlight better scaling method per model number
    for bar_idx, bar in enumerate(g_evaluation_score.patches[:12]):
        if bar.get_height() > g_evaluation_score.patches[bar_idx + 12].get_height():
            g_evaluation_score.patches[bar_idx + 12].set_alpha(0.3)
        elif bar.get_height() < g_evaluation_score.patches[bar_idx + 12].get_height():
            bar.set_alpha(0.3)
    """

    if separate_plots:
        # Save figure
        plt.savefig(f'../../plots/{approach}/{n_chunks}_chunks/{style}/rnn_results_compare_scaling_methods_{parameter}{figure_suffix}.pdf', dpi=300, bbox_inches='tight')
        plt.show(fig)

if not separate_plots:
    # Save figure
    plt.savefig(f'../../plots/{approach}/{n_chunks}_chunks/{style}/rnn_results_compare_scaling_methods{figure_suffix}.pdf', dpi=300, bbox_inches='tight')
    plt.show(fig)

In [ ]:
display(plotdata[['ID', 'TP', 'FN', 'FP', 'TN', 'FPR', 'TPR', 'FNR', 'TNR', 'WEIGHTED_SCORE_AA']])

### Compare Model Types

In [ ]:
analysis_data = result_matrix_models.copy()

for model_type in ['RNN', 'LSTM', 'GRU']:
    for parameter in ['HR', 'BP', 'O2']:
        df = analysis_data[(analysis_data['PARAMETER'] == parameter) & (analysis_data['MODEL'] == model_type)]
        print(f'{parameter}, {model_type}: Score is {df["WEIGHTED_SCORE_AA"].mean()}')

for model_type in ['RNN', 'LSTM', 'GRU']:
    df = analysis_data[analysis_data['MODEL'] == model_type]
    print(f'{model_type}: Score is {df["WEIGHTED_SCORE_AA"].mean()}')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plotdata = result_matrix_models.copy()
plotdata = plotdata.replace(['_RN|_LS|_GR'], [''], regex=True)

best_model_type_indices = pd.DataFrame(plotdata.groupby(['ID'])['WEIGHTED_SCORE_AA'].idxmax())
winning_model_types = plotdata[plotdata.index.isin(best_model_type_indices['WEIGHTED_SCORE_AA'].tolist())][['ID', 'WEIGHTED_SCORE_AA']]
print(winning_model_types)

plotdata['TEMPORARY_SORTING_COLUMN'] = plotdata['ID'] + '_'
plotdata['TEMPORARY_SORTING_COLUMN'] = plotdata.TEMPORARY_SORTING_COLUMN.replace(['_H_', '_L_'], ['', ''], regex=True)
plotdata = plotdata.sort_values(by=['TEMPORARY_SORTING_COLUMN', 'ALARM_TYPE'], ascending = [True, False]).drop(['TEMPORARY_SORTING_COLUMN'], axis = 1).reset_index(drop=True)

if not separate_plots:
    sns.set_style('whitegrid')
    fig, axs = plt.subplots(
        nrows=nr,
        ncols=nc,
        figsize=fs,
        dpi=72
        )
    fig.subplots_adjust(hspace=0.6, wspace=0.4)

    evaluation_score_ylimits = [0, result_matrix_models.WEIGHTED_SCORE_AA.max(skipna=True) + 0.05]

for i, parameter in enumerate(available_parameters):

    if separate_plots:
        # Setup plot
        sns.set_style('whitegrid')
        fig, ax = plt.subplots(
            nrows=1,
            ncols=1,
            dpi=72
            )
        fig.subplots_adjust(hspace=0.6, wspace=0.4)

        evaluation_score_ylimits = [0, plotdata[plotdata.PARAMETER == parameter].WEIGHTED_SCORE_AA.max(skipna=True) + 0.05]
        axes = ax

    else:
        axes = axs[i]

    g_evaluation_score = sns.barplot(
        ax=axes,
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='WEIGHTED_SCORE_AA',
        hue='MODEL',
        palette=sns.color_palette('colorblind')[2:],
        ci=None)
    axes.set_title(parameter_labels[parameter], fontweight='bold', color='black', fontsize=14)
    axes.set_ylim(evaluation_score_ylimits)
    axes.tick_params(axis='x', rotation=90)
    axes.set_ylabel('Evaluation Score')
    axes.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, title='Model Type: ')

    """
    # Highlight better model type
    for bar_idx, bar in enumerate(g_evaluation_score.patches[:12]):
        heights_competing_model_types = [bar.get_height(),
                                         g_evaluation_score.patches[bar_idx + 12].get_height(),
                                         g_evaluation_score.patches[bar_idx + 24].get_height()]
        idx_max_height = heights_competing_model_types.index(max(heights_competing_model_types))

        if idx_max_height == 0:
            g_evaluation_score.patches[bar_idx + 12].set_alpha(0.3)
            g_evaluation_score.patches[bar_idx + 24].set_alpha(0.3)
        elif idx_max_height == 1:
            bar.set_alpha(0.3)
            g_evaluation_score.patches[bar_idx + 24].set_alpha(0.3)
        else:
            bar.set_alpha(0.3)
            g_evaluation_score.patches[bar_idx + 12].set_alpha(0.3)
    """

    if separate_plots:
        # Save figure
        plt.savefig(f'../../plots/{approach}/{n_chunks}_chunks/{style}/rnn_results_compare_model_types_{parameter}{figure_suffix}.pdf', dpi=300, bbox_inches='tight')
        plt.show(fig)

if not separate_plots:
    # Save figure
    plt.savefig(f'../../plots/{approach}/{n_chunks}_chunks/{style}/rnn_results_compare_model_types{figure_suffix}.pdf', dpi=300, bbox_inches='tight')
    plt.show(fig)

In [ ]:
display(plotdata[['ID', 'TP', 'FN', 'FP', 'TN', 'FPR', 'TPR', 'FNR', 'TNR', 'WEIGHTED_SCORE_AA']])

### Compare Best Scaled with Non-Scaled Models

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Extract IDs of scaled models with better evaluation score
plotdata_scaled = result_matrix_models[result_matrix_models['SCALING'] != np.nan]
plotdata_scaled['ID_TEMP'] = plotdata_scaled['ID']
plotdata_scaled['ID_TEMP'] = plotdata_scaled['ID_TEMP'].replace(['_s1', '_s2'], ['', ''], regex=True)

best_scaling_indices = pd.DataFrame(plotdata_scaled.groupby(['ID_TEMP'])['WEIGHTED_SCORE_AA'].idxmax())
best_scaling_ids = plotdata_scaled[plotdata_scaled.index.isin(best_scaling_indices['WEIGHTED_SCORE_AA'].tolist())][['ID']]
print(best_scaling_ids)

# Filter result matrix and adjust IDs
plotdata = result_matrix_models[(result_matrix_models['ID'].isin(best_scaling_ids['ID'].tolist())) |
                                (result_matrix_models['SCALING'] == np.nan)]
plotdata = plotdata.replace(['_s1|_s2|_n'], [''], regex=True)
plotdata['SCALING'] = plotdata['SCALING'].replace([np.nan, 'Standard|Min-Max'], ['False', 'True'], regex=True)

plotdata['TEMPORARY_SORTING_COLUMN'] = plotdata['ID'] + '_'
plotdata['TEMPORARY_SORTING_COLUMN'] = plotdata.TEMPORARY_SORTING_COLUMN.replace(['_H_', '_L_'], ['', ''], regex=True)
plotdata = plotdata.sort_values(by=['TEMPORARY_SORTING_COLUMN', 'ALARM_TYPE'], ascending = [True, False]).drop(['TEMPORARY_SORTING_COLUMN'], axis = 1).reset_index(drop=True)

if not separate_plots:
    # Setup plot
    sns.set_style('whitegrid')
    fig, axs = plt.subplots(
        nrows=nr,
        ncols=nc,
        figsize=fs,
        dpi=72
        )
    fig.subplots_adjust(hspace=0.6, wspace=0.4)

    evaluation_score_ylimits = [0, result_matrix_models.WEIGHTED_SCORE_AA.max(skipna=True) + 0.05]

# Actual plot per parameter
for i, parameter in enumerate(available_parameters):

    if separate_plots:
        # Setup plot
        sns.set_style('whitegrid')
        fig, ax = plt.subplots(
            nrows=1,
            ncols=1,
            dpi=72
            )
        fig.subplots_adjust(hspace=0.6, wspace=0.4)

        evaluation_score_ylimits = [0, plotdata[plotdata.PARAMETER == parameter].WEIGHTED_SCORE_AA.max(skipna=True) + 0.05]
        axes = ax

    else:
        axes = axs[i]

    g_evaluation_score = sns.barplot(
        ax=axes,
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='WEIGHTED_SCORE_AA',
        hue='SCALING',
        palette=sns.color_palette('colorblind')[2:],
        ci=None)
    axes.set_title(parameter_labels[parameter], fontweight='bold', color='black', fontsize=14)
    axes.set_ylim(evaluation_score_ylimits)
    axes.tick_params(axis='x', rotation=90)
    axes.set_ylabel('Evaluation Score')
    axes.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, title='Scaled: ')

    """
    # Highlight better model type
    for bar_idx, bar in enumerate(g_evaluation_score.patches[:12]):
        if bar.get_height() > g_evaluation_score.patches[bar_idx + 12].get_height():
            g_evaluation_score.patches[bar_idx + 12].set_alpha(0.3)
        else:
            bar.set_alpha(0.3)
    """

    if separate_plots:
        # Save figure
        plt.savefig(f'../../plots/{approach}/{n_chunks}_chunks/{style}/rnn_results_compare_scaled_and_not_{parameter}{figure_suffix}.pdf', dpi=300, bbox_inches='tight')
        plt.show(fig)

if not separate_plots:
    # Save figure
    plt.savefig(f'../../plots/{approach}/{n_chunks}_chunks/{style}/rnn_results_compare_scaled_and_not{figure_suffix}.pdf', dpi=300, bbox_inches='tight')
    plt.show(fig)

In [ ]:
display(plotdata[['ID', 'TP', 'FN', 'FP', 'TN', 'FPR', 'TPR', 'FNR', 'TNR', 'WEIGHTED_SCORE_AA']])

### Compare Both Scaling Methods with Non-Scaled Models

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Adjust ID and scaling method columns
plotdata = result_matrix_models.copy()
plotdata['ID'] = plotdata['ID'].replace(['_s1|_s2|_n'], [''], regex=True)
plotdata['SCALING'] = plotdata['SCALING'].replace([np.nan], ['None'], regex=True)

# Sort (Low first)
plotdata['TEMPORARY_SORTING_COLUMN'] = plotdata['ID'] + '_'
plotdata['TEMPORARY_SORTING_COLUMN'] = plotdata.TEMPORARY_SORTING_COLUMN.replace(['_H_', '_L_'], ['', ''], regex=True)
plotdata = plotdata.sort_values(by=['TEMPORARY_SORTING_COLUMN', 'ALARM_TYPE'], ascending = [True, False]).drop(['TEMPORARY_SORTING_COLUMN'], axis = 1).reset_index(drop=True)

if not separate_plots:
    # Setup plot
    sns.set_style('whitegrid')
    fig, axs = plt.subplots(
        nrows=nr,
        ncols=nc,
        figsize=fs,
        dpi=72
        )
    fig.subplots_adjust(hspace=0.6, wspace=0.4)

    evaluation_score_ylimits = [0, result_matrix_models.WEIGHTED_SCORE_AA.max(skipna=True) + 0.05]

# Actual plot per parameter
for i, parameter in enumerate(available_parameters):

    if separate_plots:
        # Setup plot
        sns.set_style('whitegrid')
        fig, ax = plt.subplots(
            nrows=1,
            ncols=1,
            dpi=72
            )
        fig.subplots_adjust(hspace=0.6, wspace=0.4)

        evaluation_score_ylimits = [0, plotdata[plotdata.PARAMETER == parameter].WEIGHTED_SCORE_AA.max(skipna=True) + 0.05]
        axes = ax

    else:
        axes = axs[i]

    g_evaluation_score = sns.barplot(
        ax=axes,
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='WEIGHTED_SCORE_AA',
        hue='SCALING',
        palette=sns.color_palette('colorblind')[2:],
        ci=None)
    axes.set_title(parameter_labels[parameter], fontweight='bold', color='black', fontsize=14)
    axes.set_ylim(evaluation_score_ylimits)
    axes.tick_params(axis='x', rotation=90)
    axes.set_ylabel('Evaluation Score')
    axes.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, title='Scaling: ')

    """
    # Highlight better model type
    for bar_idx, bar in enumerate(g_evaluation_score.patches[:12]):
        heights_competing_model_types = [bar.get_height(),
                                         g_evaluation_score.patches[bar_idx + 12].get_height(),
                                         g_evaluation_score.patches[bar_idx + 24].get_height()]
        idx_max_height = heights_competing_model_types.index(max(heights_competing_model_types))

        if idx_max_height == 0:
            g_evaluation_score.patches[bar_idx + 12].set_alpha(0.3)
            g_evaluation_score.patches[bar_idx + 24].set_alpha(0.3)
        elif idx_max_height == 1:
            bar.set_alpha(0.3)
            g_evaluation_score.patches[bar_idx + 24].set_alpha(0.3)
        else:
            bar.set_alpha(0.3)
            g_evaluation_score.patches[bar_idx + 12].set_alpha(0.3)
    """

    if separate_plots:
        # Save figure
        plt.savefig(f'../../plots/{approach}/{n_chunks}_chunks/{style}/rnn_results_compare_both_scalings_with_non_scaled_{parameter}{figure_suffix}.pdf', dpi=300, bbox_inches='tight')
        plt.show(fig)

if not separate_plots:
    # Save figure
    plt.savefig(f'../../plots/{approach}/{n_chunks}_chunks/{style}/rnn_results_compare_both_scalings_with_non_scaled{figure_suffix}.pdf', dpi=300, bbox_inches='tight')
    plt.show(fig)

In [ ]:
display(plotdata[['ID', 'TP', 'FN', 'FP', 'TN', 'FPR', 'TPR', 'FNR', 'TNR', 'WEIGHTED_SCORE_AA']])

### Plot Final Model Selection

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plotdata = result_matrix_models.copy()
plotdata['ID'] = plotdata['ID'] + '_'
plotdata['ID'] = plotdata['ID'].replace(['_H_', '_L_'], ['', ''], regex=True)
plotdata.sort_values(by=['ID'], inplace=True)

if not separate_plots:
    # Setup plot
    sns.set_style('whitegrid')
    fig, axs = plt.subplots(
        nrows=nr,
        ncols=nc,
        figsize=fs,
        dpi=72
        )
    fig.subplots_adjust(hspace=0.6, wspace=0.4)

    evaluation_score_ylimits = [0, result_matrix_models.WEIGHTED_SCORE_AA.max(skipna=True) + 0.05]

# Actual plot per parameter
for i, parameter in enumerate(available_parameters):

    print(f'\n##### {parameter} #####')

    if separate_plots:
        # Setup plot
        sns.set_style('whitegrid')
        fig, ax = plt.subplots(
            nrows=1,
            ncols=1,
            dpi=72
            )
        fig.subplots_adjust(hspace=0.6, wspace=0.4)

        evaluation_score_ylimits = [0, plotdata[plotdata.PARAMETER == parameter].WEIGHTED_SCORE_AA.max(skipna=True) + 0.05]
        axes = ax

    else:
        axes = axs[i]

    g_evaluation_score = sns.barplot(
        ax=axes,
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='WEIGHTED_SCORE_AA',
        hue='ALARM_TYPE',
        palette=sns.color_palette('colorblind'),
        ci=None,
        hue_order=['Low', 'High'])
    axes.set_title(parameter_labels[parameter], fontweight='bold', color='black', fontsize=14)
    axes.set_ylim(evaluation_score_ylimits)
    axes.tick_params(axis='x', rotation=90)
    axes.set_ylabel('Evaluation Score')
    axes.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

    # Show models with best WS_AA
    best_weighted_aa_score_low = plotdata[plotdata.WEIGHTED_SCORE_AA == plotdata[(plotdata.PARAMETER == parameter) & (plotdata.ALARM_TYPE == 'Low')].WEIGHTED_SCORE_AA.max()]
    display(best_weighted_aa_score_low[['ID', 'TP', 'FN', 'FP', 'TN', 'FPR', 'TPR', 'FNR', 'TNR', 'WEIGHTED_SCORE_AA']])
    best_weighted_aa_score_high = plotdata[plotdata.WEIGHTED_SCORE_AA == plotdata[(plotdata.PARAMETER == parameter) & (plotdata.ALARM_TYPE == 'High')].WEIGHTED_SCORE_AA.max()]
    display(best_weighted_aa_score_high[['ID', 'TP', 'FN', 'FP', 'TN', 'FPR', 'TPR', 'FNR', 'TNR', 'WEIGHTED_SCORE_AA']])

    # Highlight models with best WS_AA
    for bar in g_evaluation_score.patches:
        if bar.get_height() != best_weighted_aa_score_low['WEIGHTED_SCORE_AA'].unique() and bar.get_height() != best_weighted_aa_score_high['WEIGHTED_SCORE_AA'].unique() :
            bar.set_alpha(0.3)

    if separate_plots:
        # Save figure
        plt.savefig(f'../../plots/{approach}/{n_chunks}_chunks/{style}/rnn_results_final_model_selection_{parameter}{figure_suffix}.pdf', dpi=300, bbox_inches='tight')
        plt.show(fig)

if not separate_plots:
    # Save figure
    plt.savefig(f'../../plots/{approach}/{n_chunks}_chunks/{style}/rnn_results_final_model_selection{figure_suffix}.pdf', dpi=300, bbox_inches='tight')
    plt.show(fig)

### Plot Runtimes

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

plotdata = result_matrix_models[result_matrix_models['ALARM_TYPE'] == 'High']
plotdata['ID'] = plotdata['ID'].replace(['_H'], [''], regex=True)

# Create subplots
sns.set_style('whitegrid')
fig, axs = plt.subplots(
    nrows=nr,
    ncols=nc,
    figsize=fs,
    dpi=72
    )

# Add main title
#plt.suptitle(f'Runtime of Predictions with {n_chunks} Chunks ({style.replace("_", " ").upper()})', fontsize=22)

# Add actual plot and adjust texts
for i, parameter in enumerate(available_parameters):

    sns.barplot(
        ax=axs[i],
        data=plotdata[plotdata.PARAMETER == parameter],
        x='ID',
        y='RUNTIME',
        color=sns.color_palette('colorblind')[2],
        ci=None)
    axs[i].set_title(parameter_labels[parameter], fontweight='bold', color='black', fontsize=14)
    axs[i].set_xticklabels(axs[i].get_xticklabels(), rotation=90)
    axs[i].set_yticklabels(pd.to_datetime(axs[i].get_yticks(), unit='s').strftime('%d Days %H:%M:%S'))

# Improve layout and save figure
fig.tight_layout()
plt.show(fig)
fig.savefig(f'../../plots/{approach}/{n_chunks}_chunks/{style}/rnn_results_runtimes{figure_suffix}.pdf', dpi=300)